In [1]:
import pickle
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

from sklearn import metrics

In [2]:
train_f = pd.read_pickle('../dataset/train/train_all_feature.pkl')
train_f.shape

(879539, 367)

In [3]:
test_f = pd.read_pickle('../dataset/train/test_all_feature.pkl')
test_f.shape

(215287, 367)

In [4]:
model_path = '../dataset/lightgbmmodel.pkl'

In [5]:
luck = 123456

In [6]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [7]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
#     'max_depth': [2, 6, 12],
    'max_depth': [12],
    'num_leaves': range(30, 50, 4), 

#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [9]:
use_columns = [
    x for x in train_f.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel', 'index'
    ]
]

In [10]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [11]:
clf_feture = train_f.sample(100000, random_state=luck)

In [12]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 5 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.6s finished


CPU times: user 15.8 s, sys: 718 ms, total: 16.5 s
Wall time: 26.2 s


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [12], 'num_leaves': range(30, 50, 4)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [13]:
clf.best_score_

0.97435000000000005

In [14]:
clf.best_params_

{'max_depth': 12, 'num_leaves': 30}

In [15]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8

clf.best_params_['colsample_bylevel'] = 0.7

# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 56


# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1
clf.best_params_['lambda_l2'] = 10
clf.best_params_['tree_method'] = 'exact'
clf.best_params_['metric'] = 'binary_logloss'



In [32]:
params = {
    'boosting_type': 'gbdt',
#     'objective': 'binary',
    'metric': 'binary_logloss',
    'min_child_weight': 1.5,
    'num_leaves': 2 ** 5,
    'lambda_l2': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
#     'colsample_bylevel': 0.7,
    'learning_rate': 0.05,
#     'tree_method': 'exact',
}

In [17]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1300,
    **params)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7, lambda_l2=10,
        learning_rate=0.05, max_bin=255, max_depth=-1,
        metric='binary_logloss', min_child_samples=10,
        min_child_weight=1.5, min_split_gain=0, n_estimators=1300,
        nthread=4, num_leaves=32, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=123456, silent=False, subsample=0.7,
        subsample_for_bin=50000, subsample_freq=1)

In [41]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel']),
             (test_f[use_columns],
    test_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=20,
)

KeyboardInterrupt: 

In [ ]:
def get_score(model, train_f):
    result = model.predict_proba(train_f[use_columns])[:, 1]
    train_f['prob'] = result
    train_x = train_f[['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [ ]:
train_score = get_score(model, train_f)

In [ ]:
test_score = get_score(model, test_f)

In [ ]:
acu = model.score(train_f[use_columns], train_f['orderlabel'])
acu

In [ ]:
# y_predict = model.predict_proba(train_features[use_columns])[:, 1]
# fpr, tpr, thresholds  = metrics.roc_curve(train_features['orderlabel'], y_predict, pos_label=1)
# auc_score = metrics.auc(fpr, tpr)
# auc_score

In [ ]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

In [ ]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [ ]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

In [ ]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in params.items()])
parms

In [ ]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    end='\n',
    sep='  ')

In [ ]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.best_score['valid_0']['auc'],
    'model_test',
    model.best_score['valid_1']['auc'],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    end='\n',
    sep='  ', file=open('lgb_result.txt', 'a+'))

In [ ]:
pickle.dump(model, open(model_path, 'wb'))

In [ ]:
# clf1 =  LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=42, max_depth=6)
# clf2 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=32, max_depth=5)
# clf3 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=27, max_depth=3)
# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
#                           meta_classifier=lr)

# model = sclf